In [2]:
#卷积神经网络我们一定要熟悉每一层过后的维度
# 98 = 100 - 3 + 1
# 有 10*5*3*3 个要优化的权重
# 有10个卷积核，每个卷积核的channel与input_channel相同，是5，卷积核的长宽是kernel_size
import torch
in_channels, out_channels= 5, 10 
width, height = 100, 100 
kernel_size = 3
batch_size = 1
input = torch.randn(batch_size, in_channels,
                    width,
                    height)
conv_layer = torch.nn.Conv2d(in_channels, out_channels,
                             kernel_size=kernel_size)
output = conv_layer(input)
print(input.shape) 
print(output.shape) 
print(conv_layer.weight.shape)

torch.Size([1, 5, 100, 100])
torch.Size([1, 10, 98, 98])
torch.Size([10, 5, 3, 3])


In [4]:
# padding
import torch
input = [3,4,6,5,7, 2,4,6,8,2, 1,6,7,8,4, 9,7,4,6,2, 3,7,5,4,1]
input = torch.Tensor(input).view(1, 1, 5, 5)
conv_layer = torch.nn.Conv2d(1, 1, kernel_size=3, padding=1, bias=False)
kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1, 1, 3, 3)
conv_layer.weight.data = kernel.data
output = conv_layer(input) 
print(output)

tensor([[[[ 91., 168., 224., 215., 127.],
          [114., 211., 295., 262., 149.],
          [192., 259., 282., 214., 122.],
          [194., 251., 253., 169.,  86.],
          [ 96., 112., 110.,  68.,  31.]]]], grad_fn=<ThnnConv2DBackward>)


In [5]:
# stride
import torch
input = [3,4,6,5,7, 2,4,6,8,2, 1,6,7,8,4, 9,7,4,6,2, 3,7,5,4,1]
input = torch.Tensor(input).view(1, 1, 5, 5)
conv_layer = torch.nn.Conv2d(1, 1, kernel_size=3, stride=2, bias=False)
kernel = torch.Tensor([1,2,3,4,5,6,7,8,9]).view(1, 1, 3, 3)
conv_layer.weight.data = kernel.data
output = conv_layer(input)
print(output)

tensor([[[[211., 262.],
          [251., 169.]]]], grad_fn=<ThnnConv2DBackward>)


In [6]:
# 最大池化
import torch
input = [3,4,6,5, 2,4,6,8, 1,6,7,8, 9,7,4,6,
        ]
input = torch.Tensor(input).view(1, 1, 4, 4)
maxpooling_layer = torch.nn.MaxPool2d(kernel_size=2)
output = maxpooling_layer(input)
print(output)

tensor([[[[4., 8.],
          [9., 8.]]]])


In [12]:
# 平均池化
import torch
input = [3,4,6,5, 2,4,6,8, 1,6,7,8, 9,7,4,6,
        ]
input = torch.Tensor(input).view(1, 1, 4, 4)
maxpooling_layer = torch.nn.AvgPool2d(kernel_size = 2)
output = maxpooling_layer(input)
print(output)

tensor([[[[3.2500, 6.2500],
          [5.7500, 6.2500]]]])


In [9]:
# 对于pytorch view函数
a=torch.Tensor([[[1,2,3],[4,5,6]]])
b=torch.Tensor([1,2,3,4,5,6])

print(a.view(1,6))
print(b.view(1,6))
print(a.view(2,-1))   # -1会自动帮忙算

tensor([[1., 2., 3., 4., 5., 6.]])
tensor([[1., 2., 3., 4., 5., 6.]])
tensor([[1., 2., 3.],
        [4., 5., 6.]])


In [11]:
# 卷积神经网络
import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
 
# prepare dataset
 
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
 
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
 
# design model using class
 
 
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = torch.nn.Conv2d(10, 20, kernel_size=5)
        self.pooling = torch.nn.MaxPool2d(2)
        self.fc = torch.nn.Linear(320, 10)
 
 
    def forward(self, x):
        # flatten data from (n,1,28,28) to (n, 784)
        # mnist是黑白图，只有一个channel
        batch_size = x.size(0)
        x = F.relu(self.pooling(self.conv1(x)))
        x = F.relu(self.pooling(self.conv2(x)))
        x = x.view(batch_size, -1) # -1 此处自动算出的是320
        # print(x.shape) 此处自动酸楚是320
        #print(batch_size)是64
        x = self.fc(x)
        # 后面用的CrossENtropyLoss，即使是分类也不需要经过softmax
        return x
 
 
model = Net()
 
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
 
# training cycle forward, backward, update
 
 
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
 
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
 
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0
 
 
def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('accuracy on test set: %d %% ' % (100*correct/total))
 
 
if __name__ == '__main__':
    for epoch in range(10):
        train(epoch)
        test()

torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size([64, 320])
torch.Size

KeyboardInterrupt: 